In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce

In [34]:
untable1 = pd.read_csv('data/UN_MigrantStockTotal_2015_Table1.csv')
untable2 = pd.read_csv('data/UN_MigrantStockTotal_2015_Table2.csv')
untable3 = pd.read_csv('data/UN_MigrantStockTotal_2015_Table3.csv')
untable4 = pd.read_csv('data/UN_MigrantStockTotal_2015_Table4.csv')
untable5 = pd.read_csv('data/UN_MigrantStockTotal_2015_Table5.csv')
untable6 = pd.read_csv('data/UN_MigrantStockTotal_2015_Table6.csv')
untableannex = pd.read_csv('data/UN_MigrantStockTotal_2015_ANNEX.csv')

In [35]:
untable1.head(20)

Unnamed: 0                                         Unnamed: 1 Unnamed: 2  \
0           NaN                                                NaN        NaN   
1           NaN                                                NaN        NaN   
2           NaN                                                NaN        NaN   
3           NaN                                                NaN        NaN   
4           NaN                                                NaN        NaN   
5           NaN                                                NaN        NaN   
6           NaN                                                NaN        NaN   
7           NaN                                                NaN        NaN   
8           NaN                                                NaN        NaN   
9           NaN                                                NaN        NaN   
10          NaN                                                NaN        NaN   
11          NaN                                                NaN        NaN   
12          NaN                                                NaN        NaN   
13  Sort\norder  Major area, region, country or area of destina...      Notes   
14          NaN                                                NaN        NaN   
15            1                                              WORLD        NaN   
16            2                                  Developed regions        (b)   
17            3                                 Developing regions        (c)   
18            4                          Least developed countries        (d)   
19            5  Less developed regions excluding least develop...        NaN   

      Unnamed: 3                                         Unnamed: 4  \
0            NaN                                                NaN   
1            NaN                                                NaN   
2            NaN                                                NaN   
3            NaN                                     United Nations   
4            NaN                                Population Division   
5            NaN          Department of Economic and Social Affairs   
6            NaN                                                NaN   
7            NaN  Trends in International Migrant Stock: The 201...   
8            NaN  Table 1 -  International migrant stock at mid-...   
9            NaN                          POP/DB/MIG/Stock/Rev.2015   
10           NaN  December 2015 - Copyright © 2015 by United Nat...   
11           NaN  Suggested citation: United Nations, Department...   
12           NaN                                                NaN   
13  Country code                                   Type of data (a)   
14           NaN                                                NaN   
15           900                                                NaN   
16           901                                                NaN   
17           902                                                NaN   
18           941                                                NaN   
19           934                                                NaN   

                                           Unnamed: 5    Unnamed: 6  \
0                                                 NaN           NaN   
1                                                 NaN           NaN   
2                                                 NaN           NaN   
3                                                 NaN           NaN   
4                                                 NaN           NaN   
5                                                 NaN           NaN   
6                                                 NaN           NaN   
7                                                 NaN           NaN   
8                                                 NaN           NaN   
9                                                 NaN           NaN   
10                            

In [36]:
#function to tidy table
def tidytable(df, x):
    #rename columns
    df.columns = ['Destination', 'Notes', 'Country code', 'Type of data', 'B1990', 'B1995', 'B2000', 'B2005', 'B2010', 'B2015', 'M1990', 'M1995', 'M2000', 'M2005', 'M2010', 'M2015', 'F1990', 'F1995', 'F2000', 'F2005', 'F2010', 'F2015']
    #drop rows containing column name information
    df_tmp = df.drop([13,14])
    #clean columns with gender and year as column names
    df_tmp = df_tmp.melt(id_vars = ['Destination', 'Notes', 'Country code', 'Type of data'], var_name = 'GendernYear', value_name = "")
    df_tmp = df_tmp.rename(columns = {"" : x})
    #split year column with gender + year information to two columns
    df_tidy = (df_tmp.assign(Gender = lambda x: x.GendernYear.str[0].astype(str),
                           Year = lambda x: x.GendernYear.str[1:].astype(str))
              .drop('GendernYear', axis=1))
    #rename gender columne
    df_tidy.loc[(df_tidy.Gender == "B"), "Gender"] = "Both"
    df_tidy.loc[(df_tidy.Gender == "M"), "Gender"] = "Male"
    df_tidy.loc[(df_tidy.Gender == "F"), "Gender"] = "Female"
    #rearrange columne orders
    col = ['Destination', 'Notes', 'Country code', 'Type of data', 'Year', 'Gender', x]
    df_tidy = df_tidy.reindex(columns=col)
    return df_tidy

In [37]:
#tidy table 1, using columns and iloc to drop the decorative table headers
untable1_tidy = tidytable(untable1[untable1.columns[1:23]].iloc[13:], 'International Migrant Stock')
untable1_tidy.head(10)

Destination Notes Country code  \
0                                              WORLD   NaN          900   
1                                  Developed regions   (b)          901   
2                                 Developing regions   (c)          902   
3                          Least developed countries   (d)          941   
4  Less developed regions excluding least develop...   NaN          934   
5                                 Sub-Saharan Africa   (e)          947   
6                                             Africa   NaN          903   
7                                     Eastern Africa   NaN          910   
8                                            Burundi   NaN          108   
9                                            Comoros   NaN          174   

  Type of data  Year Gender International Migrant Stock  
0          NaN  1990   Both                 152 563 212  
1          NaN  1990   Both                  82 378 628  
2          NaN  1990   Both                  70 184 584  
3          NaN  1990   Both                  11 075 966  
4          NaN  1990   Both                  59 105 261  
5          NaN  1990   Both                  14 690 319  
6          NaN  1990   Both                  15 690 623  
7          NaN  1990   Both                   5 964 031  
8          B R  1990   Both                     333 110  
9            B  1990   Both                      14 079

In [38]:
#tidy table 2
#adding Type of data (a) column to table 2 to align with other tables
typeofdata_df = untable1['Unnamed: 4']
typeofdata_df.columns = ['typeofdata']
untable2 = untable2.join(typeofdata_df, how='left', rsuffix='typeofdata')
cols = untable2.columns.tolist()
cols

['Unnamed: 0',
 'Unnamed: 1',
 'Unnamed: 2',
 'Unnamed: 3',
 'Unnamed: 4',
 'Unnamed: 5',
 'Unnamed: 6',
 'Unnamed: 7',
 'Unnamed: 8',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 'Unnamed: 18',
 'Unnamed: 19',
 'Unnamed: 20',
 'Unnamed: 21',
 'Unnamed: 4typeofdata']

In [39]:
cols_tmp = untable2.columns.tolist()
cols_tmp
untable2 = untable2[['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 4typeofdata','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11','Unnamed: 12','Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16','Unnamed: 17','Unnamed: 18','Unnamed: 19','Unnamed: 20','Unnamed: 21']]
untable2.head(20)

Unnamed: 0                                         Unnamed: 1 Unnamed: 2  \
0           NaN                                                NaN        NaN   
1           NaN                                                NaN        NaN   
2           NaN                                                NaN        NaN   
3           NaN                                                NaN        NaN   
4           NaN                                                NaN        NaN   
5           NaN                                                NaN        NaN   
6           NaN                                                NaN        NaN   
7           NaN                                                NaN        NaN   
8           NaN                                                NaN        NaN   
9           NaN                                                NaN        NaN   
10          NaN                                                NaN        NaN   
11          NaN                                                NaN        NaN   
12          NaN                                                NaN        NaN   
13  Sort\norder  Major area, region, country or area of destina...      Notes   
14          NaN                                                NaN        NaN   
15            1                                              WORLD        NaN   
16            2                                  Developed regions        (b)   
17            3                                 Developing regions        (c)   
18            4                          Least developed countries        (d)   
19            5  Less developed regions excluding least develop...        NaN   

      Unnamed: 3                               Unnamed: 4typeofdata  \
0            NaN                                                NaN   
1            NaN                                                NaN   
2            NaN                                                NaN   
3            NaN                                     United Nations   
4            NaN                                Population Division   
5            NaN          Department of Economic and Social Affairs   
6            NaN                                                NaN   
7            NaN  Trends in International Migrant Stock: The 201...   
8            NaN  Table 1 -  International migrant stock at mid-...   
9            NaN                          POP/DB/MIG/Stock/Rev.2015   
10           NaN  December 2015 - Copyright © 2015 by United Nat...   
11           NaN  Suggested citation: United Nations, Department...   
12           NaN                                                NaN   
13  Country code                                   Type of data (a)   
14           NaN                                                NaN   
15           900                                                NaN   
16           901                                                NaN   
17           902                                                NaN   
18           941                                                NaN   
19           934                                                NaN   

                                           Unnamed: 4  Unnamed: 5  Unnamed: 6  \
0                                                 NaN         NaN         NaN   
1                                                 NaN         NaN         NaN   
2                                                 NaN         NaN         NaN   
3                                      United Nations         NaN         NaN   
4                                 Population Division         NaN         NaN   
5           Department of Economic and Social Affairs         NaN         NaN   
6                                                 NaN         NaN         NaN   
7   Trends in International Migrant Stock: The 201...         NaN         NaN   
8   Table 2 -  Total population at mid-year by sex...         NaN         NaN   
9

In [40]:
#tidy table 2, using columns and iloc to drop the decorative table headers
untable2_tidy = tidytable(untable2[untable2.columns[1:23]].iloc[13:], 'Population (thousands)')
untable2_tidy.head(10)

Destination Notes Country code  \
0                                              WORLD   NaN          900   
1                                  Developed regions   (b)          901   
2                                 Developing regions   (c)          902   
3                          Least developed countries   (d)          941   
4  Less developed regions excluding least develop...   NaN          934   
5                                 Sub-Saharan Africa   (e)          947   
6                                             Africa   NaN          903   
7                                     Eastern Africa   NaN          910   
8                                            Burundi   NaN          108   
9                                            Comoros   NaN          174   

  Type of data  Year Gender Population (thousands)  
0          NaN  1990   Both              5 309 668  
1          NaN  1990   Both              1 144 463  
2          NaN  1990   Both              4 165 205  
3          NaN  1990   Both                510 058  
4          NaN  1990   Both              3 655 147  
5          NaN  1990   Both                491 498  
6          NaN  1990   Both                631 614  
7          NaN  1990   Both                198 232  
8          B R  1990   Both                  5 613  
9            B  1990   Both                    415

In [41]:
#tidy table 3, using columns and iloc to drop the decorative table headers
untable3_tidy = tidytable(untable3[untable3.columns[1:23]].iloc[13:], 'International Migrant Stock as a Percentage of the Population')
untable3_tidy.head(10)

Destination Notes Country code  \
0                                              WORLD   NaN          900   
1                                  Developed regions   (b)          901   
2                                 Developing regions   (c)          902   
3                          Least developed countries   (d)          941   
4  Less developed regions excluding least develop...   NaN          934   
5                                 Sub-Saharan Africa   (e)          947   
6                                             Africa   NaN          903   
7                                     Eastern Africa   NaN          910   
8                                            Burundi   NaN          108   
9                                            Comoros   NaN          174   

  Type of data  Year Gender  \
0          NaN  1990   Both   
1          NaN  1990   Both   
2          NaN  1990   Both   
3          NaN  1990   Both   
4          NaN  1990   Both   
5          NaN  1990   Both   
6          NaN  1990   Both   
7          NaN  1990   Both   
8          B R  1990   Both   
9            B  1990   Both   

  International Migrant Stock as a Percentage of the Population  
0                                                2.9             
1                                                7.2             
2                                                1.7             
3                                                2.2             
4                                                1.6             
5                                                3.0             
6                                                2.5             
7                                                3.0             
8                                                5.9             
9                                                3.4

In [42]:
#tidy table 4
#seperate function to tidy table 4 as only female information is included in the table
def tidytable4(df, x):
    #rename columns
    df.columns = ['Destination', 'Notes', 'Country code', 'Type of data', 'F1990', 'F1995', 'F2000', 'F2005', 'F2010', 'F2015']
    #drop rows containing column name information
    df_tmp = df.drop([13,14])
    #clean columns with gender and year as column names
    df_tmp = df_tmp.melt(id_vars = ['Destination', 'Notes', 'Country code', 'Type of data'], var_name = 'GendernYear', value_name = "")
    df_tmp = df_tmp.rename(columns = {"" : x})
    #split year column with gender + year information to two columns
    df_tidy = (df_tmp.assign(Gender = lambda x: x.GendernYear.str[0].astype(str),
                           Year = lambda x: x.GendernYear.str[1:].astype(str))
              .drop('GendernYear', axis=1))
    #rename gender columne
    df_tidy.loc[(df_tidy.Gender == "F"), "Gender"] = "Female"
    #rearrange columne orders
    col = ['Destination', 'Notes', 'Country code', 'Type of data', 'Year', 'Gender', x]
    df_tidy = df_tidy.reindex(columns=col)
    return df_tidy
untable4_tidy = tidytable4(untable4[untable4.columns[1:23]].iloc[13:], 'Migrants as a percentage of the International migrant stock')
untable4_tidy.head(10)

Destination Notes Country code  \
0                                              WORLD   NaN          900   
1                                  Developed regions   (b)          901   
2                                 Developing regions   (c)          902   
3                          Least developed countries   (d)          941   
4  Less developed regions excluding least develop...   NaN          934   
5                                 Sub-Saharan Africa   (e)          947   
6                                             Africa   NaN          903   
7                                     Eastern Africa   NaN          910   
8                                            Burundi   NaN          108   
9                                            Comoros   NaN          174   

  Type of data  Year  Gender  \
0          NaN  1990  Female   
1          NaN  1990  Female   
2          NaN  1990  Female   
3          NaN  1990  Female   
4          NaN  1990  Female   
5          NaN  1990  Female   
6          NaN  1990  Female   
7          NaN  1990  Female   
8          B R  1990  Female   
9            B  1990  Female   

  Migrants as a percentage of the International migrant stock  
0                                               49.0           
1                                               51.1           
2                                               46.6           
3                                               47.3           
4                                               46.5           
5                                               47.3           
6                                               47.2           
7                                               48.5           
8                                               51.0           
9                                               52.3

In [43]:
#tidy table 5
#seperate function to tidy table 5 as it involves year-year range
def tidytable5(df, x):
    #rename columns
    df.columns = ['Destination', 'Notes', 'Country code', 'Type of data', 'B1990-1995', 'B1995-2000', 'B2000-2005', 'B2005-2010', 'B2010-2015', 'M1990-1995', 'M1995-2000', 'M2000-2005', 'M2005-2010', 'M2010-2015', 'F1990-1995', 'F1995-2000', 'F2000-2005', 'F2005-2010', 'F2010-2015']
    #drop rows containing column name information
    df_tmp = df.drop([13,14])
    #clean columns with gender and year as column names
    df_tmp = df_tmp.melt(id_vars = ['Destination', 'Notes', 'Country code', 'Type of data'], var_name = 'GendernYear', value_name = "")
    df_tmp = df_tmp.rename(columns = {"" : x})
    #split year column with gender + year information to two columns
    df_tidy = (df_tmp.assign(Gender = lambda x: x.GendernYear.str[0].astype(str),
                           Year = lambda x: x.GendernYear.str[1:].astype(str))
              .drop('GendernYear', axis=1))
    #rename gender columne
    df_tidy.loc[(df_tidy.Gender == "B"), "Gender"] = "Both"
    df_tidy.loc[(df_tidy.Gender == "M"), "Gender"] = "Male"
    df_tidy.loc[(df_tidy.Gender == "F"), "Gender"] = "Female"
    #rearrange columne orders
    col = ['Destination', 'Notes', 'Country code', 'Type of data', 'Year', 'Gender', x]
    df_tidy = df_tidy.reindex(columns=col)
    return df_tidy
untable5_tidy = tidytable5(untable5[untable5.columns[1:20]].iloc[13:], 'Annual rate of change of the migrant stock')
untable5_tidy.head(10)

Destination Notes Country code  \
0                                              WORLD   NaN          900   
1                                  Developed regions   (b)          901   
2                                 Developing regions   (c)          902   
3                          Least developed countries   (d)          941   
4  Less developed regions excluding least develop...   NaN          934   
5                                 Sub-Saharan Africa   (e)          947   
6                                             Africa   NaN          903   
7                                     Eastern Africa   NaN          910   
8                                            Burundi   NaN          108   
9                                            Comoros   NaN          174   

  Type of data       Year Gender Annual rate of change of the migrant stock  
0          NaN  1990-1995   Both                                       1.05  
1          NaN  1990-1995   Both                                       2.28  
2          NaN  1990-1995   Both                                      -0.49  
3          NaN  1990-1995   Both                                       1.12  
4          NaN  1990-1995   Both                                      -0.80  
5          NaN  1990-1995   Both                                       0.85  
6          NaN  1990-1995   Both                                       0.83  
7          NaN  1990-1995   Both                                      -3.44  
8          B R  1990-1995   Both                                      -5.36  
9            B  1990-1995   Both                                      -0.20

In [44]:
#tidy table 6
def tidytable6(df):
    #rename columns
    df.columns = ['Destination', 'Notes', 'Country code', 'Type of data', 'B1990', 'B1995', 'B2000', 'B2005', 'B2010', 'B2015', 'B1990', 'B1995', 'B2000', 'B2005', 'B2010', 'B2015', 'B1990-1995', 'B1995-2000', 'B2000-2005', 'B2005-2010', 'B2010-2015']
    #drop rows containing column name information
    df = df.drop([13, 14])
    return df

#function to tidy table 6 subsets
def tidysubset6(df, x):
    #clean columns with gender and year as column names
    df_tmp = df.melt(id_vars = ['Destination', 'Notes', 'Country code', 'Type of data'], var_name = 'GendernYear', value_name = "")
    df_tmp = df_tmp.rename(columns = {"" : x})
    #split year column with gender + year information to two columns
    df_tidy = (df_tmp.assign(Gender = lambda x: x.GendernYear.str[0].astype(str),
                           Year = lambda x: x.GendernYear.str[1:].astype(str))
              .drop('GendernYear', axis=1))
    #rename gender columne
    df_tidy.loc[(df_tidy.Gender == "B"), "Gender"] = "Both"
    #rearrange columne orders
    col = ['Destination', 'Notes', 'Country code', 'Type of data', 'Year', 'Gender', x]
    df_tidy = df_tidy.reindex(columns=col)
    return df_tidy


untable6_tmp = tidytable6(untable6[untable6.columns[1:22]].iloc[13:]) 
untable6_tmp.head(20)
untable6_subset1 = tidysubset6(untable6_tmp.iloc[:279, :10], 'Estimated refugee stock at mid-year')
untable6_subset2 = tidysubset6(untable6_tmp.iloc[:279, [0,1,2,3,10,11,12,13,14,15]], 'Refugees as a percentage of the international migrant stock')
untable6_subset3 = tidysubset6(untable6_tmp.iloc[:279, [0,1,2,3,16,17,18,19,20]], 'Annual rate of change of the refugee stock')
untable6_subset1.head(10)

Destination Notes Country code  \
0                                              WORLD   NaN          900   
1                                  Developed regions   (b)          901   
2                                 Developing regions   (c)          902   
3                          Least developed countries   (d)          941   
4  Less developed regions excluding least develop...   NaN          934   
5                                 Sub-Saharan Africa   (e)          947   
6                                             Africa   NaN          903   
7                                     Eastern Africa   NaN          910   
8                                            Burundi   NaN          108   
9                                            Comoros   NaN          174   

  Type of data  Year Gender Estimated refugee stock at mid-year  
0          NaN  1990   Both                          18 836 571  
1          NaN  1990   Both                           2 014 564  
2          NaN  1990   Both                          16 822 007  
3          NaN  1990   Both                           5 048 391  
4          NaN  1990   Both                          11 773 616  
5          NaN  1990   Both                           5 516 042  
6          NaN  1990   Both                           5 687 352  
7          NaN  1990   Both                           3 168 001  
8          B R  1990   Both                             267 929  
9            B  1990   Both                                   0

In [45]:
untable6_subset2.head(10)

Destination Notes Country code  \
0                                              WORLD   NaN          900   
1                                  Developed regions   (b)          901   
2                                 Developing regions   (c)          902   
3                          Least developed countries   (d)          941   
4  Less developed regions excluding least develop...   NaN          934   
5                                 Sub-Saharan Africa   (e)          947   
6                                             Africa   NaN          903   
7                                     Eastern Africa   NaN          910   
8                                            Burundi   NaN          108   
9                                            Comoros   NaN          174   

  Type of data  Year Gender  \
0          NaN  1990   Both   
1          NaN  1990   Both   
2          NaN  1990   Both   
3          NaN  1990   Both   
4          NaN  1990   Both   
5          NaN  1990   Both   
6          NaN  1990   Both   
7          NaN  1990   Both   
8          B R  1990   Both   
9            B  1990   Both   

  Refugees as a percentage of the international migrant stock  
0                                               12.3           
1                                                2.4           
2                                               24.0           
3                                               45.6           
4                                               19.9           
5                                               37.5           
6                                               36.2           
7                                               53.1           
8                                               80.4           
9                                                0.0

In [46]:
untable6_subset3.head(10)

Destination Notes Country code  \
0                                              WORLD   NaN          900   
1                                  Developed regions   (b)          901   
2                                 Developing regions   (c)          902   
3                          Least developed countries   (d)          941   
4  Less developed regions excluding least develop...   NaN          934   
5                                 Sub-Saharan Africa   (e)          947   
6                                             Africa   NaN          903   
7                                     Eastern Africa   NaN          910   
8                                            Burundi   NaN          108   
9                                            Comoros   NaN          174   

  Type of data       Year Gender Annual rate of change of the refugee stock  
0          NaN  1990-1995   Both                                      -2.12  
1          NaN  1990-1995   Both                                       9.39  
2          NaN  1990-1995   Both                                      -2.84  
3          NaN  1990-1995   Both                                      -0.68  
4          NaN  1990-1995   Both                                      -4.38  
5          NaN  1990-1995   Both                                      -0.02  
6          NaN  1990-1995   Both                                       0.08  
7          NaN  1990-1995   Both                                      -5.31  
8          B R  1990-1995   Both                                      -3.39  
9            B  1990-1995   Both                                         ..

In [47]:
#merge table with migrant and refugee information against year
untable_year = [untable2_tidy, untable1_tidy, untable6_subset1, untable3_tidy, untable4_tidy, untable6_subset2]
untable_year_tidy= reduce(lambda left,right: pd.merge(left,right,on=['Destination', 'Notes', 'Country code', 'Type of data', 'Year', 'Gender'], how='left'), untable_year)
untable_year_tidy.sample(10)

Destination Notes Country code Type of data  Year  Gender  \
1282                 Ecuador   NaN          218          B R  2010    Both   
636                Sri Lanka   NaN          144          B R  2000    Both   
3710                   WORLD   NaN          900          NaN  2000  Female   
4720                  Panama   NaN          591            B  2015  Female   
2537         Southern Europe   NaN          925          NaN  2005    Male   
3113              Montserrat   NaN          500            B  2015    Male   
1327      Developing regions   (c)          902          NaN  2015    Both   
566                    Gabon   NaN          266            C  2000    Both   
4379         Channel Islands  (14)          830            B  2010  Female   
3897  British Virgin Islands   NaN           92            B  2000  Female   

     Population (thousands) International Migrant Stock  \
1282                 14 935                     325 366   
636                  18 784                      40 132   
3710              3 042 084                  84 818 470   
4720                  1 959                      89 869   
2537                 73 302                   5 832 012   
3113                     ..                         696   
1327              6 098 121                 103 218 281   
566                   1 232                     195 571   
4379                     81                      40 599   
3897                     ..                       6 116   

     Estimated refugee stock at mid-year  \
1282                             121 249   
636                                   16   
3710                                 NaN   
4720                                 NaN   
2537                                 NaN   
3113                                 NaN   
1327                          17 623 250   
566                               17 982   
4379                                 NaN   
3897                                 NaN   

     International Migrant Stock as a Percentage of the Population  \
1282                                                2.2              
636                                                 0.2              
3710                                                2.8              
4720                                                4.6              
2537                                                8.0              
3113                                                 ..              
1327                                                1.7              
566                                                15.9              
4379                                               50.4              
3897                                                 ..              

     Migrants as a percentage of the International migrant stock  \
1282                                                NaN            
636                                                 NaN            
3710                                               49.1            
4720                                               48.7            
2537                                                NaN            
3113                                                NaN            
1327                                                NaN            
566                                                 NaN            
4379                                               52.3            
3897                                               50.7            

     Refugees as a percentage of the international migrant stock  
1282                                               37.3           
636                                                 0.0           
3710                                                NaN           
4720                                                NaN           
2537                                                NaN           
3113                                                NaN           
1327                              

In [48]:
#merge table with migrant and refugee information against year-year range
untable_range = [untable5_tidy, untable6_subset3]
untable_range_tidy= reduce(lambda left,right: pd.merge(left,right,on=['Destination', 'Notes', 'Country code', 'Type of data', 'Year', 'Gender'], how='left'), untable_range)
untable_range_tidy.sample(10)

Destination Notes Country code Type of data  \
3029                            Israel   NaN          376          B R   
3003                         Indonesia   NaN          360          B R   
3933       Falkland Islands (Malvinas)   NaN          238            B   
2096                         Melanesia   NaN          928          NaN   
3314                           Romania   NaN          642            B   
807                           Ethiopia   NaN          231          B R   
2826                            Monaco   NaN          492            B   
584                       Burkina Faso   NaN          854          B R   
1013  Bolivia (Plurinational State of)   NaN           68            B   
1054                              Niue   NaN          570            B   

           Year  Gender Annual rate of change of the migrant stock  \
3029  1995-2000  Female                                       0.73   
3003  1995-2000  Female                                      -5.97   
3933  2010-2015  Female                                       2.08   
2096  2000-2005    Male                                       2.14   
3314  2000-2005  Female                                       3.41   
807   2005-2010    Both                                       1.98   
2826  1990-1995  Female                                       0.39   
584   2000-2005    Both                                       2.76   
1013  2005-2010    Both                                       2.62   
1054  2005-2010    Both                                       0.86   

     Annual rate of change of the refugee stock  
3029                                        NaN  
3003                                        NaN  
3933                                        NaN  
2096                                        NaN  
3314                                        NaN  
807                                        6.53  
2826                                        NaN  
584                                       -5.14  
1013                                       3.64  
1054                                         ..

In [49]:
#tidy untableannex
untableannex_tmp = untableannex[untableannex.columns[0:23]].iloc[13:]
untableannex_tmp.columns = untableannex_tmp.iloc[0]
untableannex_tmp = untableannex_tmp.drop([13])
untableannex_tmp = untableannex_tmp.drop('Sort order', axis=1)
#rename columns
untableannex_tmp.columns = ['Country code', 'Country or area', 'Major area', 'Major area code', 'Region', 'Region code', 'Developed region', 'Least developed country', 'Sub-Saharan Africa']
cols_an = untableannex_tmp.columns.tolist()
cols_an

['Country code',
 'Country or area',
 'Major area',
 'Major area code',
 'Region',
 'Region code',
 'Developed region',
 'Least developed country',
 'Sub-Saharan Africa']

In [50]:
#rearrange columns
untableannex_tmp = untableannex_tmp[['Country or area', 'Major area', 'Region', 'Country code', 'Major area code', 'Region code','Developed region', 'Least developed country', 'Sub-Saharan Africa']].reset_index()
#get a seperate df for destination information
notes = untable1_tidy[untable1_tidy.columns[1:3]].iloc[:]
#tidy ANNEX table
untableannex_tidy = untableannex_tmp.merge(notes, on=['Country code']).drop_duplicates()
untableannex_tidy.head(20)

index      Country or area                       Major area  \
0       14          Afghanistan                             Asia   
18      15              Albania                           Europe   
36      16              Algeria                           Africa   
54      17       American Samoa                          Oceania   
72      18              Andorra                           Europe   
90      19               Angola                           Africa   
108     20             Anguilla  Latin America and the Caribbean   
126     21  Antigua and Barbuda  Latin America and the Caribbean   
144     22            Argentina  Latin America and the Caribbean   
162     23              Armenia                             Asia   
180     24                Aruba  Latin America and the Caribbean   
198     25            Australia                          Oceania   
216     26              Austria                           Europe   
234     27           Azerbaijan                             Asia   
252     28              Bahamas  Latin America and the Caribbean   
270     29              Bahrain                             Asia   
288     30           Bangladesh                             Asia   
306     31             Barbados  Latin America and the Caribbean   
324     32              Belarus                           Europe   
342     33              Belgium                           Europe   

                        Region Country code Major area code Region code  \
0                Southern Asia            4             935        5501   
18             Southern Europe            8             908         925   
36             Northern Africa           12             903         912   
54                   Polynesia           16             909         957   
72             Southern Europe           20             908         925   
90               Middle Africa           24             903         911   
108                  Caribbean          660             904         915   
126                  Caribbean           28             904         915   
144              South America           32             904         931   
162               Western Asia           51             935         922   
180                  Caribbean          533             904         915   
198  Australia and New Zealand           36             909         927   
216             Western Europe           40             908         926   
234               Western Asia           31             935         922   
252                  Caribbean           44             904         915   
270               Western Asia           48             935         922   
288              Southern Asia           50             935        5501   
306                  Caribbean           52             904         915   
324             Eastern Europe          112             908         923   
342             Western Europe           56             908         926   

    Developed region Least developed country Sub-Saharan Africa Notes  
0                 No                     Yes                 No   NaN  
18               Yes                      No                 No   NaN  
36                No                      No                 No   NaN  
54                No                      No                 No   NaN  
72               Yes                      No                 No   NaN  
90                No                     Yes                Yes   NaN  
108               No                      No                 No   NaN  
126               No                      No                 No   NaN  
144               No                      No                 No   NaN  
162               No                      No                 No   NaN  
180               No                      No                 No   NaN  
198              Yes                      No                 No  (22)  
216              Yes                      No                 No 

In [51]:
#function for seperating the df into region and country data
def regiondata(df):
    df_region = df[df['Type of data'].isna()]
    #df_region = df_region.drop(['Destination', 'Notes'], axis=1)
    return df_region

def countrydata(df):
    df_country = df[df['Type of data'].notna()]
    #df_country = df_country.drop(['Destination', 'Notes'], axis=1)
    return df_country

In [52]:
#seperating region and country data to two tables
#table on migrant/refugee stock against year
untable_year_region = regiondata(untable_year_tidy)
untable_year_country = countrydata(untable_year_tidy)
untable_year_region.head(10)

Destination Notes Country code  \
0                                               WORLD   NaN          900   
1                                   Developed regions   (b)          901   
2                                  Developing regions   (c)          902   
3                           Least developed countries   (d)          941   
4   Less developed regions excluding least develop...   NaN          934   
5                                  Sub-Saharan Africa   (e)          947   
6                                              Africa   NaN          903   
7                                      Eastern Africa   NaN          910   
28                                      Middle Africa   NaN          911   
38                                    Northern Africa   NaN          912   

   Type of data  Year Gender Population (thousands)  \
0           NaN  1990   Both              5 309 668   
1           NaN  1990   Both              1 144 463   
2           NaN  1990   Both              4 165 205   
3           NaN  1990   Both                510 058   
4           NaN  1990   Both              3 655 147   
5           NaN  1990   Both                491 498   
6           NaN  1990   Both                631 614   
7           NaN  1990   Both                198 232   
28          NaN  1990   Both                 70 886   
38          NaN  1990   Both                140 117   

   International Migrant Stock Estimated refugee stock at mid-year  \
0                  152 563 212                          18 836 571   
1                   82 378 628                           2 014 564   
2                   70 184 584                          16 822 007   
3                   11 075 966                           5 048 391   
4                   59 105 261                          11 773 616   
5                   14 690 319                           5 516 042   
6                   15 690 623                           5 687 352   
7                    5 964 031                           3 168 001   
28                   1 460 530                             446 609   
38                   2 403 200                           1 202 360   

   International Migrant Stock as a Percentage of the Population  \
0                                                 2.9              
1                                                 7.2              
2                                                 1.7              
3                                                 2.2              
4                                                 1.6              
5                                                 3.0              
6                                                 2.5              
7                                                 3.0              
28                                                2.1              
38                                                1.7              

   Migrants as a percentage of the International migrant stock  \
0                                                 NaN            
1                                                 NaN            
2                                                 NaN            
3                                                 NaN            
4                                                 NaN            
5                                                 NaN            
6                                                 NaN            
7                                                 NaN            
28                                                NaN            
38                                                NaN            

   Refugees as a percentage of the international migrant stock  
0                                                12.3           
1                                                 2.4           
2                                                24.0           
3                                                45.6           
4                          

In [53]:
untable_year_country.head(10)

Destination Notes Country code Type of data  Year Gender  \
8      Burundi   NaN          108          B R  1990   Both   
9      Comoros   NaN          174            B  1990   Both   
10    Djibouti   NaN          262          B R  1990   Both   
11     Eritrea   NaN          232            I  1990   Both   
12    Ethiopia   NaN          231          B R  1990   Both   
13       Kenya   NaN          404          B R  1990   Both   
14  Madagascar   NaN          450            C  1990   Both   
15      Malawi   NaN          454          B R  1990   Both   
16   Mauritius   (1)          480            C  1990   Both   
17     Mayotte   NaN          175            B  1990   Both   

   Population (thousands) International Migrant Stock  \
8                   5 613                     333 110   
9                     415                      14 079   
10                    588                     122 221   
11                  3 139                      11 848   
12                 48 057                   1 155 390   
13                 23 446                     297 292   
14                 11 546                      23 917   
15                  9 409                   1 127 724   
16                  1 056                       3 613   
17                     95                      15 229   

   Estimated refugee stock at mid-year  \
8                              267 929   
9                                    0   
10                              54 508   
11                                   0   
12                             741 965   
13                              13 452   
14                                   0   
15                             874 614   
16                                   0   
17                                   0   

   International Migrant Stock as a Percentage of the Population  \
8                                                 5.9              
9                                                 3.4              
10                                               20.8              
11                                                0.4              
12                                                2.4              
13                                                1.3              
14                                                0.2              
15                                               12.0              
16                                                0.3              
17                                               16.1              

   Migrants as a percentage of the International migrant stock  \
8                                                 NaN            
9                                                 NaN            
10                                                NaN            
11                                                NaN            
12                                                NaN            
13                                                NaN            
14                                                NaN            
15                                                NaN            
16                                                NaN            
17                                                NaN            

   Refugees as a percentage of the international migrant stock  
8                                                80.4           
9                                                 0.0           
10                                               44.6           
11                                                0.0           
12                                               64.2           
13                                                4.5           
14                                                0.0           
15                                               77.6           
16                                                0.0           
17                                                0.0

In [54]:
#seperating region and country data to two tables
#table on migrant/refugee stock against year range+
untable_range_region = regiondata(untable_range_tidy)
untable_range_country = countrydata(untable_range_tidy)
untable_range_region.head(10)

Destination Notes Country code  \
0                                               WORLD   NaN          900   
1                                   Developed regions   (b)          901   
2                                  Developing regions   (c)          902   
3                           Least developed countries   (d)          941   
4   Less developed regions excluding least develop...   NaN          934   
5                                  Sub-Saharan Africa   (e)          947   
6                                              Africa   NaN          903   
7                                      Eastern Africa   NaN          910   
28                                      Middle Africa   NaN          911   
38                                    Northern Africa   NaN          912   

   Type of data       Year Gender Annual rate of change of the migrant stock  \
0           NaN  1990-1995   Both                                       1.05   
1           NaN  1990-1995   Both                                       2.28   
2           NaN  1990-1995   Both                                      -0.49   
3           NaN  1990-1995   Both                                       1.12   
4           NaN  1990-1995   Both                                      -0.80   
5           NaN  1990-1995   Both                                       0.85   
6           NaN  1990-1995   Both                                       0.83   
7           NaN  1990-1995   Both                                      -3.44   
28          NaN  1990-1995   Both                                      11.89   
38          NaN  1990-1995   Both                                      -2.87   

   Annual rate of change of the refugee stock  
0                                       -2.12  
1                                        9.39  
2                                       -2.84  
3                                       -0.68  
4                                       -4.38  
5                                       -0.02  
6                                        0.08  
7                                       -5.31  
28                                      12.96  
38                                      -3.46

In [55]:
untable_range_country.head(10)

Destination Notes Country code Type of data       Year Gender  \
8      Burundi   NaN          108          B R  1990-1995   Both   
9      Comoros   NaN          174            B  1990-1995   Both   
10    Djibouti   NaN          262          B R  1990-1995   Both   
11     Eritrea   NaN          232            I  1990-1995   Both   
12    Ethiopia   NaN          231          B R  1990-1995   Both   
13       Kenya   NaN          404          B R  1990-1995   Both   
14  Madagascar   NaN          450            C  1990-1995   Both   
15      Malawi   NaN          454          B R  1990-1995   Both   
16   Mauritius   (1)          480            C  1990-1995   Both   
17     Mayotte   NaN          175            B  1990-1995   Both   

   Annual rate of change of the migrant stock  \
8                                       -5.36   
9                                       -0.20   
10                                      -4.06   
11                                       0.91   
12                                      -7.18   
13                                      14.66   
14                                      -2.43   
15                                     -30.81   
16                                      14.59   
17                                      10.94   

   Annual rate of change of the refugee stock  
8                                       -3.39  
9                                          ..  
10                                      -9.76  
11                                         ..  
12                                      -5.51  
13                                      42.52  
14                                         ..  
15                                    -104.31  
16                                         ..  
17                                         ..